In [2]:
import os
import getpass

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake

import dotenv
dotenv.load_dotenv()

os.environ['ACTIVELOOP_TOKEN'] = "YourAPIKey" # replace with your API key from app.activeloop.ai
username = "rihp" # replace with your username from app.activeloop.ai
projectname = "autogpt1" # replace with your project name from app.activeloop.ai


In [3]:
embeddings = OpenAIEmbeddings(disallowed_special=())

In [4]:
# !git clone https://github.com/Significant-Gravitas/Auto-GPT # replace any repository of your choice 

In [5]:
import os
from langchain.document_loaders import TextLoader

root_dir = './Auto-GPT' # Specify where your data is stored
docs = []
for dirpath, dirnames, filenames in os.walk(root_dir):
    for file in filenames:
        try: 
            loader = TextLoader(os.path.join(dirpath, file), encoding='utf-8')
            docs.extend(loader.load_and_split())
        except Exception as e: 
            pass

In [6]:
%%capture

from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(docs)

In [7]:
%%capture
db = DeepLake(dataset_path=f"hub://{username}/{projectname}", embedding_function=embeddings, public=True) #dataset would be publicly available
db.add_documents(texts)

In [8]:
db = DeepLake(dataset_path=f"hub://{username}/{projectname}", read_only=True, embedding_function=embeddings)

/

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/rihp/autogpt1



\

hub://rihp/autogpt1 loaded successfully.

Deep Lake Dataset in hub://rihp/autogpt1 already exists, loading from the storage
Dataset(path='hub://rihp/autogpt1', read_only=True, tensors=['embedding', 'ids', 'metadata', 'text'])

  tensor     htype      shape      dtype  compression
  -------   -------    -------    -------  ------- 
 embedding  generic  (920, 1536)  float32   None   
    ids      text     (920, 1)      str     None   
 metadata    json     (920, 1)      str     None   
   text      text     (920, 1)      str     None   


In [9]:
retriever = db.as_retriever()
retriever.search_kwargs['distance_metric'] = 'cos'
retriever.search_kwargs['fetch_k'] = 100
retriever.search_kwargs['maximal_marginal_relevance'] = True
retriever.search_kwargs['k'] = 10

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

model = ChatOpenAI(model_name='gpt-3.5-turbo') # switch to 'gpt-4'
qa = ConversationalRetrievalChain.from_llm(model, retriever=retriever)

In [12]:
questions = [
    """What tests have been developed for this repository? Check the tests folder for more information.""",  
    "What happens in the memory module of the repository?",
    "Can you explain how the AutoGPT repository works?",
] 
chat_history = []

for question in questions:  
    result = qa({"question": question, "chat_history": chat_history})
    chat_history.append((question, result['answer']))
    print(f"-> **Question**: {question} \n")
    print(f"**Answer**: {result['answer']} \n")

-> **Question**: What tests have been developed for this repository? Check the tests folder for more information. 

**Answer**: I'm sorry, but I cannot see any mention of a tests folder in the provided context. The context includes some code snippets, a documentation checklist, and a few references to GitHub users, but no mention of a tests folder. 

-> **Question**: What happens in the memory module of the repository? 

**Answer**: The memory module of the repository refers to the memory backend that Auto-GPT uses to store and access relevant data for its AI tasks. This module provides different options for memory backends such as local, Pinecone, Redis, Milvus, and Weaviate. Memory pre-seeding is a technique that is mentioned to improve AI accuracy by ingesting relevant data into its memory. Chunks of data are split and added to memory, allowing the AI to access them quickly and generate more accurate responses. The memory module is a crucial component of Auto-GPT as it helps improve